In [1]:
import SDS_Environment as SDS
from SDS_Environment import np, plt, tf, Camera, deque
# %matplotlib notebook

In [3]:
def initializer(env, N):
    # ---------------------------------------------------------------------------------------
    # print first line in log file
    
#     f.close()
    f = open("log_C.txt", "w")
    f.write(f"episode \t epsilon \t ")
    for i in range(N): f.write(f"r{i} \t ")
    for i in range(N): f.write(f"k{i} \t ")
    for i in range(N): f.write(f"k{i}**2 \t ")
    for i in range(N): f.write(f"k{i}**3 \t ")
    for i in range(N): f.write(f"r{i}**2 \t ")
    for i in range(N): f.write(f"fourth{i} \t ")
    for i in range(N): f.write(f"H{i} \t ")
    for i in range(N): f.write(f"reward{i} \t ")
    f.write(f"Hamilton \t\t")
    for i in range(N): f.write(f"All Q{i} \t next Q{i} \t Q{i} \t target Q{i} \t LOSS{i} \t")   ### 
    f.write(f"\t")
    for i in range(N):
        for j in range(3): f.write(f"Grads {i} {j} \t bias {i} {j} \t ")       ###
    for i in range(N): 
        for j in range(3): f.write(f"Trainable {i} {j} \t bias {i} {j} \t ")   ###
        
    
    # ---------------------------------------------------------------------------------------
    # build models
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)
    np.random.seed(42)

    input_shape = [3]              # == env.observation_space.shape
    n_outputs = 2                  # == env.action_space.n

    model = []                     # make a model for each agent
    for i in range(N):
#         initializer = tf.keras.initializers.GlorotNormal(seed = tf.random.set_seed(42+i)) # HeNormal()
        model.append(tf.keras.models.Sequential([
            tf.keras.layers.Dense(32, activation="elu", input_shape=input_shape),
            tf.keras.layers.Dense(32, activation="elu"),
            tf.keras.layers.Dense(n_outputs, activation=custom_activation)
        ]))
        
    # ---------------------------------------------------------------------------------------
    # initilaze replay_memory & best_weights
    replay_memory = []             # A bag containing 40 recently viewed items (state, action, reward, next_state)
    best_weights  = []             # The best weights that each model will use to make the best decision
    for i in range(N):
        replay_memory.append(deque(maxlen=40))
        best_weights .append(model[i].get_weights())
        
    # ---------------------------------------------------------------------------------------
    # initilaze states and rewards from environment class    
    state, reward = env.step(np.zeros(N), 1)
    state = [[state[j][i] for j in range(input_shape[0])] for i in range(N)]
    
    return f, model, n_outputs, replay_memory, best_weights, state, reward
    
def custom_activation(x):
    # return tf.keras.backend.switch(x >= 0, x**0.5, -(-x)**0.5)
    xx = (tf.keras.backend.softplus(x))**0.5
    return -tf.keras.backend.elu(-xx + 100) + 100

In [4]:
def log_print(f, N, env, episode, epsilon, reward, H  ,  all_q, next_q, qq, target_q, LOSS, grads, trainable_variables):
    
    f.write(f"\n{episode} \t {epsilon} \t ")
    for i in range(N): f.write(f"{env.r[i]} \t ")
    for i in range(N): f.write(f"{env.k[i]} \t ")
    for i in range(N): f.write(f"{env.k[i]**2} \t ")
    for i in range(N): f.write(f"{env.k[i]**3} \t ")
    for i in range(N): f.write(f"{env.r[i]**2} \t ")
    for i in range(N): f.write(f"{env.Hamiltonian(i)[1]} \t ")
    for i in range(N): f.write(f"{env.Hamiltonian(i)[0]} \t ")
    for i in range(N): f.write(f"{reward[i]} \t ")
    f.write(f"{H} \t\t")
    
    for i in range(N): 
        f.write(f"{all_q[i].numpy().ravel()} \t ")
        f.write(f"{next_q[i].ravel()} \t ")
        f.write(f"{qq[i].numpy().ravel()} \t ")
        f.write(f"{target_q[i].ravel()} \t ")
        f.write(f"{LOSS[i]} \t ")
        
    f.write(f"\t")
    np.set_printoptions(linewidth=np.inf)
    for i in range(N):
        for grad in grads[i]: f.write(f"{grad.numpy().ravel()} \t ")
    for i in range(N):
        for variable in trainable_variables[i]: f.write(f"{variable.numpy().ravel()} \t ")  

In [5]:
def play_one_step(env, N, model, state, n_outputs, replay_memory, epsilon=0.):
    action = np.zeros(N)

    for i in range(N):
        if np.random.rand() < epsilon:
            action[i] = np.random.randint(n_outputs)
        else:
            actionnnn = model[0].predict(np.reshape(state[i], (3)).reshape(1, -1), verbose=0)[0]
            action[i] = np.argmax(actionnnn)
    
    next_state, reward = env.step((action-0.5)*2 , epsilon)
    next_state = [[next_state[j][i] for j in range(3)] for i in range(N)]
    
    for i in range(N):
        replay_memory[i].append((state[i], action[i], reward[i], next_state[i]))
    return next_state, reward

In [32]:
def Plot_H_G(episode, Hamilton, Giant):
    plt.figure(figsize=(15,5))

    plt.subplot(1,2,1)
    plt.plot(Hamilton)
    plt.text(0.7*episode, 1.1*max(Hamilton), "Min(H): %f" % (min(Hamilton)) )
    plt.text(0.7*episode, 1.2*max(Hamilton), "Arg(H): %f" % (np.argmin(Hamilton)) )
    plt.grid(alpha=0.3)

    plt.subplot(1,2,2)
    plt.plot(Giant)
    plt.grid(alpha=0.3)
    plt.show()

In [ ]:
N = 100                      # Number of agents
L = 15                       # The length of the simulation box
env = SDS.Distributed_System(N,L)

batch_size = 20
discount_rate = 0.98
file, model, n_outputs, replay_memory, best_weights, state, reward = initializer(env, N)

camera = Camera(plt.figure())


best_H   = 0
Hamilton = []               # just for plot total hamilton per episode
giant = []                  # Represents the percentage of members of the giant component 

# just for checking prolems
all_q = []; next_q = []; qq = []; target_q = []; LOSS = []; grads = []; trainable_variables = []; 
for i in range(N): all_q.append(0); next_q.append(0); qq.append(0); target_q.append(0); LOSS.append(0); grads.append(0); trainable_variables.append(0)



# ---------------------------------------------------------------------------------------
for episode in range(800):
    
    for _ in range(20):
        epsilon = max(1 - episode/600, 0.0)             # first is more random and than use greedy
        state, reward = play_one_step(env, N, model, state, n_outputs, replay_memory, epsilon)

    if episode > 50:
        for i in range(N):
            (all_q[i], next_q[i], qq[i], target_q[i], LOSS[i],
             grads[i], trainable_variables[i]) = SDS.training_step(i, model, n_outputs, replay_memory, batch_size, discount_rate)
        log_print(file, N, env, episode, epsilon, reward, H, 
                  all_q, next_q, qq, target_q, LOSS, grads, trainable_variables)    
    
     
    H = 0
    for i in range(env.N): H += env.Hamiltonian(i)        # Hamiltonian of the whole system
    Hamilton.append(H) 
    
    if H <= best_H and episode > 500:                   # find the minimum of Hamiltonian
        best_weight = model[0].get_weights()            # saving model weights for the best Hamiltonian
        best_H = H
        
      
    if episode%10 == 0:
        giant.append( env.Plot(camera, episode) )
        
    print("\rEpisode: {}, eps: {:.3f}, Min(Hamilton): {:.3f}, H: {:.3f},     R1: {:.3f}, R2: {:.3f}, R3: {:.3f}, R4: {:.3f}"
          .format(episode, epsilon, best_H, H, env.r[0], env.r[1], env.r[2], env.r[3]), end="")
    
    
anim = camera.animate(interval= 100, repeat=True, repeat_delay= 500, blit=True)
anim.save('./result/animation_C.gif')
file.close()


# ---------------------------------------------------------------------------------------
print(best_H)
Plot_H_G(episode, Hamilton, giant)



# last_width = model[0].get_weights()
# model[0].save('100agent.keras')

# tests